# Edit Distance

编辑距离是针对二个字符串（例如英文字）的差异程度的量化量测，量测方式是看至少需要多少次的处理才能将一个字符串变成另一个字符串。编辑距离可以用在自然语言处理中，例如拼写检查可以根据一个拼错的字和其他正确的字的编辑距离，判断哪一个（或哪几个）是比较可能的字。DNA也可以视为用A、C、G和T组成的字符串，因此编辑距离也用在生物信息学中，判断二个DNA的类似程度。Unix 下的 diff 及 patch 即是利用编辑距离来进行文本编辑对比的例子。

在莱文斯坦距离中，可以删除、加入、取代字符串中的任何一个字元，也是较常用的编辑距离定义，常常提到编辑距离时，指的就是莱文斯坦距离。

In [61]:
import numpy as np
import pandas as pd

In [92]:
word_1 = 'abc'
word_2 = 'abvfs'

# 初始化矩阵

In [93]:
edit_matrix = np.zeros(shape=(len(word_2)+1, len(word_1)+1), dtype='int')
edit_matrix[0,:] = [x for x in range(len(word_1)+1)]
edit_matrix[:,0] = [x for x in range(len(word_2)+1)]
df_matrix = pd.DataFrame(edit_matrix, 
                         index=['\'\''] + [x for x in word_2], 
                         columns=['\'\''] + [x for x in word_1], 
                         dtype='int'
                        )
df_matrix

,'',a,b,c
'',0,1,2,3
a,1,0,0,0
b,2,0,0,0
v,3,0,0,0
f,4,0,0,0
s,5,0,0,0


# 距离计算规则函数
1. $str(i,j)$ 的值为以下三个值最小值
    1. $str(i-1,j)+1$
    2. $str(i,j-1)+1$
    3. $str(i-1,j-1)$
2. 对于$str(i-1,j-1)$
    1. 如果此位置两个字符串一样，则$str(i,j)=str(i-1,j-1)$
    2. 如果此位置两字符串不一样，则$str(i,j)=str(i-1,j-1)+1$

In [94]:
def distance(matrix, word_1, word_2, i, j):
    # 斜上方计算 str(i-1,j-1)
    if word_2[i-1] == word_1[j-1]:
        up_left = matrix[i-1, j-1]
    else:
        up_left = matrix[i-1, j-1] + 1
    
    # 正上方计算 str(i-1,j)+1
    up = matrix[i-1, j] + 1  

    # 左方的计算 str(i,j-1)+1
    left = matrix[i, j-1] + 1  
    
    # 返回三个值最小的值
    return min(up, left, up_left)

# 计算完整矩阵

In [95]:
for i in range(1, len(word_2)+1):
    for j in range(1, len(word_1)+1):
        edit_matrix[i,j] = distance(edit_matrix, word_1, word_2, i, j)

df_matrix = pd.DataFrame(edit_matrix, 
                         index=['\'\''] + [x for x in word_2], 
                         columns=['\'\''] + [x for x in word_1], 
                         dtype='int'
                        )
df_matrix

,'',a,b,c
'',0,1,2,3
a,1,0,1,2
b,2,1,0,1
v,3,2,1,1
f,4,3,2,2
s,5,4,3,3


# 输出结果
1. 在右下角的值就是最终距离的结果

In [96]:
result = edit_matrix.ravel()[-1]
print('距离为：', result)

距离为： 3


# 函数封装

In [97]:
def edit_distance(word_1, word_2):
    # 字典保存距离矩阵
    distance_dict = {}
    
    # 初始化距离
    for i in range(len(word_2)+1):
        distance_dict[(i,0)] = i
    
    for j in range(len(word_1)+1):
        distance_dict[(0,j)] = j
    
    # 计算全部矩阵
    for i in range(1, len(word_2)+1):
        for j in range(1, len(word_1)+1):
            
            # 斜上方计算 str(i-1,j-1)
            if word_2[i-1] == word_1[j-1]:
                up_left = distance_dict[(i-1, j-1)]
            else:
                up_left = distance_dict[(i-1, j-1)] + 1

            # 正上方计算 str(i-1,j)+1
            up = distance_dict[(i-1, j)] + 1  

            # 左方的计算 str(i,j-1)+1
            left = distance_dict[(i, j-1)] + 1  
            
            # 添加到矩阵中
            distance_dict[(i, j)] = min(up, left, up_left)
    
    return distance_dict[(len(word_2)), len(word_1)]


In [98]:
edit_distance('abc', 'abvfs')

3